In [1]:
import pyspark
import pandas as pd
import pyspark.sql
import matplotlib.pyplot as plt
import seaborn as sns
import time
import findspark
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
%matplotlib inline
from pyspark.ml.feature import CountVectorizer

In [2]:
from pyspark.sql import SparkSession

In [3]:
findspark.init(r"C:\Users\rejee\spark-3.3.1-bin-hadoop3")

In [4]:
spark=SparkSession.builder.appName("Amazone").getOrCreate()

In [5]:
spark


In [6]:
df = spark.read.load(".\kindle_reviews.csv", format="csv", sep=",", header="true", inferSchema=True)

In [7]:
df.show()

+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|      asin|helpful|overall|          reviewText|          reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|B000F83SZQ| [0, 0]|      5|I enjoy vintage b...|          05 5, 2014|      A1F6404F1VG29J|          Avidreader|  Nice vintage story|          1399248000|
|  1|B000F83SZQ| [2, 2]|      4|This book is a re...|          01 6, 2014|       AN0N05A9LIJEQ|            critters|        Different...|          1388966400|
|  2|B000F83SZQ| [2, 2]|      4|This was a fairly...|          04 4, 2014|       A795DMNCJILA6|                 dot|               Oldie|          1396569600|
|  3|B000F83SZQ| [1, 1]|      5|I'd never read

In [8]:
[trainingData, testData] = df.randomSplit([0.7, 0.3])

In [9]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- helpful: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: string (nullable = true)



In [10]:
df.show()

+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|      asin|helpful|overall|          reviewText|          reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|B000F83SZQ| [0, 0]|      5|I enjoy vintage b...|          05 5, 2014|      A1F6404F1VG29J|          Avidreader|  Nice vintage story|          1399248000|
|  1|B000F83SZQ| [2, 2]|      4|This book is a re...|          01 6, 2014|       AN0N05A9LIJEQ|            critters|        Different...|          1388966400|
|  2|B000F83SZQ| [2, 2]|      4|This was a fairly...|          04 4, 2014|       A795DMNCJILA6|                 dot|               Oldie|          1396569600|
|  3|B000F83SZQ| [1, 1]|      5|I'd never read

In [11]:
df=df.drop('reviewTime','reviewerName','reviewerID','helpful','unixReviewTime')

In [12]:
df.show()

+---+----------+-------+--------------------+--------------------+
|_c0|      asin|overall|          reviewText|             summary|
+---+----------+-------+--------------------+--------------------+
|  0|B000F83SZQ|      5|I enjoy vintage b...|  Nice vintage story|
|  1|B000F83SZQ|      4|This book is a re...|        Different...|
|  2|B000F83SZQ|      4|This was a fairly...|               Oldie|
|  3|B000F83SZQ|      5|I'd never read an...|  I really liked it.|
|  4|B000F83SZQ|      4|If you like perio...|      Period Mystery|
|  5|B000F83SZQ|      4|A beautiful in-de...|              Review|
|  6|B000F83SZQ|      4|I enjoyed this on...|Nice old fashione...|
|  7|B000F83SZQ|      4|Never heard of Am...|Enjoyable reading...|
|  8|B000FA64PA|      5|Darth Maul workin...|          Darth Maul|
|  9|B000FA64PA|      4|This is a short s...|Not bad, not exce...|
| 10|B000FA64PA|      5|I think I have th...|      Audio and book|
| 11|B000FA64PA|      4|Title has nothing...|Darth Maul...the 

In [14]:
tokenizer=Tokenizer(inputCol='summary',outputCol='review_token')
stop_words=StopWordsRemover(inputCol='review_token',outputCol='review_stop')
countvector=CountVectorizer(inputCol='review_stop',outputCol='review_count')


In [15]:
from pyspark.ml.feature import StringIndexer

In [16]:
indexer = StringIndexer(inputCol="overall", outputCol="label")

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
lr = LogisticRegression(maxIter = 50, regParam = 0.01)

In [20]:

pipeline = Pipeline(stages=[tokenizer,stop_words,countvector, indexer, lr])

In [21]:
model = pipeline.fit(trainingData)

Py4JJavaError: An error occurred while calling o50.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 5.0 failed 1 times, most recent failure: Lost task 6.0 in stage 5.0 (TID 18) (DESKTOP-6UT81JN.home executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$3181/0x0000000101395840: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterato